In [74]:
#!/usr/bin/env python3
import os
import sys
import string
from Crypto.Util import number
from Crypto.Util.number import bytes_to_long, long_to_bytes
from Crypto.Cipher import AES
from binascii import hexlify
from binascii import unhexlify

from ec import *
from utils import *
#from secret import flag1, flag2

#P-256 parameters
p = 0xffffffff00000001000000000000000000000000ffffffffffffffffffffffff
a = -3
b = 0x5ac635d8aa3a93e7b3ebbd55769886bc651d06b0cc53b0f63bce3c3e27d2604b
n = 0xffffffff00000000ffffffffffffffffbce6faada7179e84f3b9cac2fc632551
curve = EllipticCurve(p,a,b, order = n)
G = ECPoint(curve, 0x6b17d1f2e12c4247f8bce6e563a440f277037d812deb33a0f4a13945d898c296, 0x4fe342e2fe1a7f9b8ee7eb4a7c0f9e162bce33576b315ececbb6406837bf51f5)

d_a = bytes_to_long(os.urandom(32))
P_a = G * d_a

printable = [ord(char.encode()) for char in string.printable]

def encrypt(msg : bytes, pubkey : ECPoint):
	x = bytes_to_long(msg)
	y = modular_sqrt(x**3 + a*x + b, p)
	m = ECPoint(curve, x, y)
	d_b = number.getRandomRange(0,n)
	return (G * d_b, m + (pubkey * d_b))

def pencrypt(msg:ECPoint, pubkey:ECPoint):
	d_b = number.getRandomRange(0,n)
	return (G * d_b, msg + (pubkey * d_b))

def decrypt(B : ECPoint, c : ECPoint, d_a : int):
	if B.inf or c.inf: return b''
	return long_to_bytes((c - (B * d_a)).x)

def loop():
	print('I will decrypt anythin as long as it does not talk about flags.')
	balance = 1024
	while True:
		print('B:', end = '')
		sys.stdout.flush()
		B_input = sys.stdin.buffer.readline().strip().decode()
		print('c:', end = '')
		sys.stdout.flush()
		c_input = sys.stdin.buffer.readline().strip().decode()
		B = ECPoint(curve, *[int(_) for _ in B_input.split(',')])
		c = ECPoint(curve, *[int(_) for _ in c_input.split(',')])
		msg = decrypt(B, c, d_a)
		if b'ENO' in msg:
			balance = -1
		else:
			balance -= 1 + len([c for c in msg if c in printable])
		if balance >= 0:
			print(hexlify(msg))
			print('balance left: %d' % balance)
		else:
			print('You cannot afford any more decryptions.')
			return


import pwn


def oldmain():
	print('My public key is:')
	print(P_a)
	print('Good luck decrypting this cipher.')
	B,c = encrypt(flag1, P_a)
	print(B)
	print(c)
	key = long_to_bytes((d_a >> (8*16)) ^ (d_a & 0xffffffffffffffffffffffffffffffff))
	enc = AES.new(key, AES.MODE_ECB)
	cipher = enc.encrypt(flag2)
	print(hexlify(cipher).decode())
	try:
		loop()
	except Exception as err:
		print(repr(err))


In [72]:
test_flag = "ENO{flaggeffsdfsdlagg}"
test_a = bytes_to_long(os.urandom(32))
test_A = G * test_a
B, c = encrypt(test_flag.encode(), test_A)
nu_c = test_A + c
print(nu_c)
c_A = decrypt(B, nu_c, test_a)
print(c_A)
nu_m_y_2 = bytes_to_long(c_A) ** 3 + a * bytes_to_long(c_A) + b
nu_m_y = modular_sqrt(nu_m_y_2, p) * (-1)
nu_m = ECPoint(curve, bytes_to_long(c_A), nu_m_y)
print(nu_m)
nu_m = nu_m - test_A
print(nu_m)
msg = long_to_bytes(nu_m.x)
print(msg)

Point(63379153082268983087676419596766291431071183002471375467266701082474365703365, 24448766636727382172074101406249882571637047981542167354047190567519432832292)
b'\x86\xe5\xfcK!\x05\xb6\x03\xab\xb2\xd6G\\7\xe0b\x9c^\xe6-\xe5\xdf\x02X\xbc\x89\x80\xeb\xb9<\xd3k'
Point(61016270953544992303396866746532803952910852473903003670035305865338809013099, 34436038883246661830715487230946913456111472007422276465041793368016589576844)
Point(25930415811709767210045133132165008857678760536139645, 4859618777402601608103635029082233285154997695920661398678174923918695172684)
b'ENO{flaggeffsdfsdlagg}'


In [47]:
c = ECPoint(curve,55512627364125854178514693309545576278144178027702971401313263567778342375544, 21483996911178583058163235542084532451351848382256587736745920418246312029439)
my_a = os.urandom(32)
my_A = G * bytes_to_long(my_a)
nu_B, nu_c = pencrypt(c, my_A)
print(nu_B)
print(nu_c)

Point(3804080823088326148287757759231307392983571478699522060709469301130149930270, 71404647177415798459113112627993106878276261622566827087301271821154369932938)
Point(10146440476196763535451219975012995443820458697211245279270205938956709091870, 36990615607108298790684233114109658180963580281580207536841761206607428040707)


In [63]:
A = ECPoint(curve, 101130525322426479027081692914659643174769712523484822159445944093855575745858, 89232179367383681748429355540821259340059086141559809779443503544531289339399)
nu_c = A + c
print(nu_c)

Point(79994196063038598358011775792130676812937004640094933917227619794235459075554, 53687371447183488707026465001377872751680831554524114547842817568524888633418)


In [78]:
nu_m_x = b'ab54c08d9c9aa13b48d07d0616a44df43750a9508c4fa321f8d55fc8e9066f6f'
nu_m_x = unhexlify(nu_m_x)
nu_m_y_2 = bytes_to_long(nu_m_x) ** 3 + a * bytes_to_long(nu_m_x) + b
nu_m_y = modular_sqrt(nu_m_y_2, p) #* (-1)

M = ECPoint(curve, bytes_to_long(nu_m_x), nu_m_y)
M = M - A
print(long_to_bytes(M.x))

m = ECPoint(curve, bytes_to_long(nu_m_x), nu_m_y)
decrypt(nu_B, m, bytes_to_long(my_a))

b'ENO{ElGam4l_1s_mult1pl1cativ3}'


b'K\xf0\x82\x85\x85i\xca\xbcH\xab\xe606\xec.\xddku7\x82\x80.jo\xdb.\x0f\xe5x`\xf4c'